In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [61]:
df = pd.read_csv('full_data.csv')

In [7]:
cols_to_remove = ['partc', 'time', 'action_file_index', 'Unnamed: 0']

df = df.drop(cols_to_remove, axis=1)

In [62]:
print df.shape
df.head(5)

(1412865, 17)


,Unnamed: 0,time,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,partc,action,action_file_index
0,0,0,2.489271,-1.397067,1.899690,0.104929,0.984947,0.137365,-0.060667,-0.333742,-0.008674,0.007192,-0.004249,0.065989,1,std,6
1,1,1,2.488154,-1.396068,1.905025,0.105682,0.984774,0.138030,-0.073340,-0.302813,-0.006666,-0.082061,0.004820,0.022905,1,std,6
2,2,2,2.489687,-1.394939,1.912298,0.106144,0.984577,0.139074,-0.090184,-0.270723,-0.024985,-0.101612,0.008953,0.002664,1,std,6
3,3,3,2.494505,-1.393874,1.922063,0.106105,0.984390,0.140421,-0.089858,-0.213139,-0.040011,-0.113307,0.020615,-0.029093,1,std,6
4,4,4,2.500577,-1.393094,1.931172,0.105709,0.984253,0.141678,-0.070132,-0.098153,-0.041089,-0.111187,0.026429,-0.034393,1,std,6


#### Creating rolling windows

In [89]:
def create_sld_df_single_exp(orig_df, window_size, analytic_functions_list):
    dfs_to_concate = []
    base_df = orig_df.drop('action', axis=1)
    for func in analytic_functions_list:
        method_to_call = getattr(base_df.rolling(window=window_size), func)
        analytic_df = method_to_call()
        analytic_df = analytic_df[window_size:]
        analytic_df.columns = [col + "_sld_" + func for col in analytic_df.columns]
        dfs_to_concate.append(analytic_df)
    
    dfs_to_concate.append(orig_df['action'][window_size:])
    return pd.concat(dfs_to_concate,axis=1)

In [104]:
def create_sliding_df(orig_df, window_size, analytic_functions_list, exps, partcs):
    dfs_to_concate = []
    cols_to_drop = ['partc', 'time', 'action_file_index', 'Unnamed: 0']
    for p in partcs:
        for e in exps:
            exp_df = orig_df.loc[(orig_df['partc'] == p) & (orig_df['action_file_index'] == e),:]
            exp_df = exp_df.drop(cols_to_drop, axis=1)
            exp_roll_df = create_sld_df_single_exp(exp_df, window_size, analytic_functions_list)
            print "finished rolling for expirement {0} and participant {1}".format(e, p)
            dfs_to_concate.append(exp_roll_df)
    return pd.concat(dfs_to_concate, axis=0, ignore_index=True) 

In [108]:
analytic_functions_list = ['mean', 'sum']
sliding_df = create_sliding_df(df, 10, analytic_functions_list, range(1,10) + range(11,17), range(1,24+1)) 
# sliding_df = create_sliding_df(df, 10, analytic_functions_list, [1,2], [1]) 

finished rolling for expirement 1 and participant 1
finished rolling for expirement 2 and participant 1
finished rolling for expirement 3 and participant 1
finished rolling for expirement 4 and participant 1
finished rolling for expirement 5 and participant 1
finished rolling for expirement 6 and participant 1
finished rolling for expirement 7 and participant 1
finished rolling for expirement 8 and participant 1
finished rolling for expirement 9 and participant 1
finished rolling for expirement 11 and participant 1
finished rolling for expirement 12 and participant 1
finished rolling for expirement 13 and participant 1
finished rolling for expirement 14 and participant 1
finished rolling for expirement 15 and participant 1
finished rolling for expirement 16 and participant 1
finished rolling for expirement 1 and participant 2
finished rolling for expirement 2 and participant 2
finished rolling for expirement 3 and participant 2
finished rolling for expirement 4 and participant 2
finish

finished rolling for expirement 7 and participant 11
finished rolling for expirement 8 and participant 11
finished rolling for expirement 9 and participant 11
finished rolling for expirement 11 and participant 11
finished rolling for expirement 12 and participant 11
finished rolling for expirement 13 and participant 11
finished rolling for expirement 14 and participant 11
finished rolling for expirement 15 and participant 11
finished rolling for expirement 16 and participant 11
finished rolling for expirement 1 and participant 12
finished rolling for expirement 2 and participant 12
finished rolling for expirement 3 and participant 12
finished rolling for expirement 4 and participant 12
finished rolling for expirement 5 and participant 12
finished rolling for expirement 6 and participant 12
finished rolling for expirement 7 and participant 12
finished rolling for expirement 8 and participant 12
finished rolling for expirement 9 and participant 12
finished rolling for expirement 11 and p

finished rolling for expirement 16 and participant 21
finished rolling for expirement 1 and participant 22
finished rolling for expirement 2 and participant 22
finished rolling for expirement 3 and participant 22
finished rolling for expirement 4 and participant 22
finished rolling for expirement 5 and participant 22
finished rolling for expirement 6 and participant 22
finished rolling for expirement 7 and participant 22
finished rolling for expirement 8 and participant 22
finished rolling for expirement 9 and participant 22
finished rolling for expirement 11 and participant 22
finished rolling for expirement 12 and participant 22
finished rolling for expirement 13 and participant 22
finished rolling for expirement 14 and participant 22
finished rolling for expirement 15 and participant 22
finished rolling for expirement 16 and participant 22
finished rolling for expirement 1 and participant 23
finished rolling for expirement 2 and participant 23
finished rolling for expirement 3 and p

In [109]:
sliding_df['attitude.roll_sld_mean'].isnull().sum()


0

In [110]:
sliding_df.shape

(1409265, 25)

In [ ]:
num_exp = range(1,10) + range(11,17)
num_partc = range(1,24+1)

In [97]:
a = df.loc[(df['partc'] == 1) & ( (df['action_file_index'] == 1) | (df['action_file_index'] == 2) ) ,:]

In [98]:
a.shape

(4209, 17)

In [102]:
a["attitude.roll"].head(20)

306427    1.528132
306428    1.527992
306429    1.527765
306430    1.516768
306431    1.493941
306432    1.476302
306433    1.455153
306434    1.441702
306435    1.443440
306436    1.443071
306437    1.434186
306438    1.412539
306439    1.366432
306440    1.298239
306441    1.222763
306442    1.137867
306443    1.059596
306444    1.002833
306445    0.955538
306446    0.923844
Name: attitude.roll, dtype: float64

In [103]:
sliding_df["attitude.roll_sld_sum"].head(20)

0     14.760320
1     14.644867
2     14.483534
3     14.265005
4     13.993827
5     13.655392
6     13.259835
7     12.820966
8     12.333064
9     11.813837
10    11.296084
11    10.819930
12    10.436949
13    10.185500
14    10.092431
15    10.185578
16    10.473816
17    10.917512
18    11.421483
19    11.927379
Name: attitude.roll_sld_sum, dtype: float64